In [1]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")

/home/berk/finreg-navigator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
collection_name = "hybrid_search_demo"

client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(
            distance=models.Distance.COSINE,
            size=384,
        ),
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

True

In [3]:
documents = [
    "Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation.",
    "Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging.",
    "Brie cheese features a soft, creamy interior surrounded by an edible white rind.",
    "This French cheese has a flowing, buttery center encased in a bloomy white crust.",
    "Fresh mozzarella pairs beautifully with ripe tomatoes and basil leaves.",
    "Classic Margherita pizza topped with tomato sauce, mozzarella, and fresh basil.",
    "Parmesan requires at least 12 months of cave aging to develop its signature sharp taste.",
    "Parmigiano-Reggiano's distinctive piquant flavor comes from extended maturation in controlled environments.",
    "Grilled cheese sandwiches are the ultimate American comfort food for cold winter days.",
    "Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich.",
]

In [4]:
import uuid

client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "dense": models.Document(
                    text=doc,
                    model="sentence-transformers/all-MiniLM-L6-v2",
                ),
                "sparse": models.Document(
                    text=doc,
                    model="Qdrant/bm25",
                ),
            },
            payload={"text": doc},
        )
        for doc in documents
    ]
)

Fetching 5 files: 100%|██████████| 5/5 [00:08<00:00,  1.76s/it]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [5]:
def dense_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model="sentence-transformers/all-MiniLM-L6-v2",
        ),
        using="dense",
        limit=3,
    )
    return response.points

In [6]:
def sparse_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="sparse",
        limit=3,
    )
    return response.points

In [7]:
queries = [
    "nutty aged cheese",
    "soft French cheese",
    "pizza ingredients",
    "a good lunch",
]

for query in queries:
    print("Query:", query)

    dense_results = dense_search(query)
    print("Dense Results:")
    for result in dense_results:
        print("\t-", result.payload["text"], result.score)

    sparse_results = sparse_search(query)
    print("Sparse Results:")
    for result in sparse_results:
        print("\t-", result.payload["text"], result.score)
    print()

Query: nutty aged cheese
Dense Results:
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 0.5829766860274936
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.47647110085156985
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 0.45055332462602254
Sparse Results:
	- Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation. 5.156332492828369
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 3.0210649967193604
	- Parmesan requires at least 12 months of cave aging to develop its signature sharp taste. 1.8819332122802734

Query: soft French cheese
Dense Results:
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 0.6242110898222957
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.6030547669172122
	- Croque Monsieur combines ha

In [8]:
def rrf_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="sparse",
                limit=3,
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="sentence-transformers/all-MiniLM-L6-v2",
                ),
                using="dense",
                limit=3,
            )
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=3,
    )
    return response.points

In [9]:
for query in queries:
    print("Query:", query)

    rrf_results = rrf_search(query)
    print("RRF Results:")
    for result in rrf_results:
        print("\t-", result.payload["text"], result.score)
    print()

Query: nutty aged cheese
RRF Results:
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 0.8333333333333333
	- Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation. 0.5
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.3333333333333333

Query: soft French cheese
RRF Results:
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 1.0
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.6666666666666666
	- Grilled cheese sandwiches are the ultimate American comfort food for cold winter days. 0.25

Query: pizza ingredients
RRF Results:
	- Classic Margherita pizza topped with tomato sauce, mozzarella, and fresh basil. 1.0
	- Fresh mozzarella pairs beautifully with ripe tomatoes and basil leaves. 0.3333333333333333
	- Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich.

In [10]:
def dbsf_search(query: str) -> list[models.ScoredPoint]:
    response = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="sparse",
                limit=3,
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="sentence-transformers/all-MiniLM-L6-v2",
                ),
                using="dense",
                limit=3,
            )
        ],
        query=models.FusionQuery(fusion=models.Fusion.DBSF),
        limit=3,
    )
    return response.points

In [11]:
for query in queries:
    print("Query:", query)

    dbsf_results = dbsf_search(query)
    print("DBSF Results:")
    for result in dbsf_results:
        print("\t-", result.payload["text"], result.score)
    print()

Query: nutty aged cheese
DBSF Results:
	- Mature Gouda cheese becomes grainy and develops a rich, buttery taste with extended aging. 1.1558481551424944
	- Aged Gouda develops a crystalline texture and nutty flavor profile after 18 months of maturation. 0.6808001019858886
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 0.4362051656143661

Query: soft French cheese
DBSF Results:
	- This French cheese has a flowing, buttery center encased in a bloomy white crust. 1.2130749154308003
	- Brie cheese features a soft, creamy interior surrounded by an edible white rind. 1.1703095844673248
	- Croque Monsieur combines ham and Gruyère in France's answer to the toasted cheese sandwich. 0.30906467923275677

Query: pizza ingredients
DBSF Results:
	- Classic Margherita pizza topped with tomato sauce, mozzarella, and fresh basil. 1.1904721101347917
	- Fresh mozzarella pairs beautifully with ripe tomatoes and basil leaves. 0.42859793795470547
	- Croque Monsieur combin